In [226]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [307]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
import ast
sys.path.append('../../..')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/data/medg/misc/semeval_2010/medical-data/DDICorpus"
outdir = 'pre-processed2/extraction/'
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.ddi_preprocess.preprocess2 import get_dataset_dataframe
from relation_extraction.data.ddi_preprocess.preprocess2 import tokenize

In [280]:
#df = get_dataset_dataframe(res('Train/DrugBank/'))
#df = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'))
#df = get_dataset_dataframe(res('Train/MedLine/'))
#df = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'))

In [308]:
df_train_drugbank = get_dataset_dataframe(res('Train/DrugBank/'), relation_extraction=True)

  1%|          | 4/569 [00:00<00:17, 31.68it/s]

total_files_to_read: 569  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Train/DrugBank/


100%|██████████| 569/569 [00:05<00:00, 95.18it/s]


In [229]:
# df_train_medline = get_dataset_dataframe(res('Train/MedLine/'), relation_extraction=True)

 13%|█▎        | 18/142 [00:00<00:00, 174.17it/s]

total_files_to_read: 142  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Train/MedLine/


100%|██████████| 142/142 [00:00<00:00, 178.37it/s]


In [281]:
df_test_drugbank = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'), relation_extraction=True)

  9%|▉         | 14/158 [00:00<00:01, 131.23it/s]

total_files_to_read: 158  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Test/test_for_ddi_extraction_task/DrugBank/


100%|██████████| 158/158 [00:01<00:00, 129.16it/s]


In [282]:
df_test_medline = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'), relation_extraction=True)

 45%|████▌     | 15/33 [00:00<00:00, 147.22it/s]

total_files_to_read: 33  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Test/test_for_ddi_extraction_task/MedLine/


100%|██████████| 33/33 [00:00<00:00, 148.95it/s]


In [283]:
len(df_test_drugbank)

4308

In [284]:
len(df_test_medline)

388

## Check for empty values

In [285]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        entity_number = row.entity_number
        e1 = entity_number[0]
        e2 = entity_number[1]
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        sentence_text = df.iloc[index].sentence_text
        tokenized_sentence = df.iloc[index].tokenized_sentence
        entity_number = df.iloc[index].entity_number
        empty_rows_array.append([index, e1, e2, sentence_text, tokenized_sentence, entity_number])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'e1', 'e2', 'sentence_text', 'tokenized_sentence', 'entity_number'])
    return empty_rows_array, new_df

In [286]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [287]:
get_empty_vals(df_test_drugbank)

([], Empty DataFrame
 Columns: [index_original, e1, e2, sentence_text, tokenized_sentence, entity_number]
 Index: [])

In [288]:
get_empty_vals(df_test_medline)

([], Empty DataFrame
 Columns: [index_original, e1, e2, sentence_text, tokenized_sentence, entity_number]
 Index: [])

#### Now can write these into csv format

In [289]:
df_test_medline.to_csv(res('pre-processed2/extraction/Test/MedLine/test-medline_partialdrugblinding.csv'), encoding='utf-8', index=False)

In [290]:
df_test_drugbank.to_csv(res('pre-processed2/extraction/Test/DrugBank/test-drugbank_partialdrugblinding.csv'), encoding='utf-8', index=False)

In [165]:
# df_train_drugbank.to_csv(res('pre-processed2/extraction/Train/DrugBank/train-drugbank_partialdrugblinding.csv'), encoding='utf-8', index=False)
# # this is only relevant to the non NER task
# df_test_drugbank.to_csv(res('pre-processed2/extraction/Test/DrugBank/test-drugbank_partialdrugblinding.csv'), encoding='utf-8', index=False)
# df_train_medline.to_csv(res('pre-processed2/extraction/Train/MedLine/train-medline_partialdrugblinding.csv'), encoding='utf-8', index=False)
# df_test_medline.to_csv(res('pre-processed2/extraction/Test/MedLine/test-medline_partialdrugblinding.csv'), encoding='utf-8', index=False)

#### Write the files into .txt format

In Train DrugBank, there is one sentence without a pair. Did ```find . -name "*.xml" | xargs grep "Drug/Laboratory Test Interactions:"``` for that

In [298]:
directory = outdir + 'Test/MedLine/'
filename = 'test-medline_partialdrugblinding'

In [299]:
df_copy = pd.read_csv(res(directory+filename+'.csv'))

In [300]:
df_copy['relation_type'].unique()

array(['int', 'none', 'effect', 'mechanism', 'advise'], dtype=object)

In [301]:
relation_dict = {'advise': 0, 'effect': 1, 'mechanism': 2, 'int': 3, 'none': 4} 

In [302]:
null_row = df_copy[df_copy["relation_type"].isnull()]

In [303]:
if null_row.empty:
    idx_null_row = None
else:
    idx_null_row = null_row.index.values[0]

In [304]:
with open(res(directory + filename+'.txt'), 'w') as outfile:
    for i in range(0, len(df_copy)):
        if idx_null_row is not None and i == idx_null_row:
            continue
        row = df_copy.iloc[i]
        relation = relation_dict[row.relation_type]
        entity_number = literal_eval(row.entity_number)
        # literal eval is needed because the tuple is being stored as a string
        e1 = entity_number[0]
        e2 = entity_number[1]
        tokenized_sentence = row.tokenized_sentence
        outfile.write(str(relation) + " " + str(e1[0]) + " " + str(e1[-1]) + " " + 
                      str(e2[0]) + " " + str(e2[-1]) + " " + tokenized_sentence + "\n")
    outfile.close()

## Combine the train/test data of Medline and Drugbank

In [305]:
directory2 = outdir + 'Test/MedLine/'
file2 = 'test-medline_partialdrugblinding'
directory1 = outdir + 'Test/DrugBank/'
file1 = 'test-drugbank_partialdrugblinding'
outfile = outdir + 'test_partialdrugblinding.txt'

In [306]:
# https://stackoverflow.com/questions/13613336/python-concatenate-text-files
filenames = [res(directory1 + file1+'.txt'), res(directory2 + file2+'.txt')]
with open(res(outfile), 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)